# Save and load forecasters

Skforecast models can be easily saved and loaded from disk using the joblib library. Two handy functions, `save_forecaster` and `load_forecaster` are available to streamline this process. See below for a simple example.

A `forecaster_id` has been included when initializing the Forecaster, this may help to identify the target of the model.

<div class="admonition note" name="html-admonition" style="background: rgba(0,184,212,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00b8d4; border-color: #00b8d4; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00b8d4;"></i>
    <b style="color: #00b8d4;">&#9998 Note</b>
</p>

Learn how to use <a href="../user_guides/forecaster-in-production.html">forecaster models in production</a>.

</div>

## Libraries and data

In [1]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

from skforecast.datasets import fetch_dataset
from skforecast.recursive import ForecasterRecursive
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

In [2]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "date"], "header": 0}
)
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('MS')

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)


## Save and load forecaster model

In [3]:
# Create and train forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor     = RandomForestRegressor(random_state=123),
                 lags          = 5,
                 forecaster_id = "forecaster_001"
             )

forecaster.fit(y=data['y'])
forecaster.predict(steps=3)

2008-07-01    0.714526
2008-08-01    0.789144
2008-09-01    0.818433
Freq: MS, Name: pred, dtype: float64

In [4]:
# Save model
# ==============================================================================
save_forecaster(forecaster, file_name='forecaster_001.joblib', verbose=False)

In [5]:
# Load model
# ==============================================================================
forecaster_loaded = load_forecaster('forecaster_001.joblib', verbose=True)

ForecasterRecursive 
Regressor: RandomForestRegressor 
Lags: [1 2 3 4 5] 
Window features: None 
Window size: 5 
Exogenous included: False 
Exogenous names: None 
Transformer for y: None 
Transformer for exog: None 
Weight function included: False 
Differentiation order: None 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth':
    None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None,
    'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100,
    'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0,
    'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-11-10 17:43:25 
Last fit date: 2024-11-10 17:43:25 
Skforecast version: 0.14.0 
Python 

In [6]:
# Predict
# ==============================================================================
forecaster_loaded.predict(steps=3)

2008-07-01    0.714526
2008-08-01    0.789144
2008-09-01    0.818433
Freq: MS, Name: pred, dtype: float64

In [7]:
# Forecaster identifier
# ==============================================================================
forecaster.forecaster_id

'forecaster_001'

## Saving and Loading a Forecaster Model with Custom Features

Sometimes external objects are needed when creating a Forecaster. For example:

+ Custom class to [create window and custom features](../user_guides/window-features-and-custom-features.html#create-your-custom-window-features).

+ A function to reduce the impact of some dates on the model, [Weighted Time Series Forecasting](../user_guides/weighted-time-series-forecasting.html). 

For your code to work properly, these functions must be available in the environment where the Forecaster is loaded.

In [8]:
# Custom class to create rolling skewness features
# ==============================================================================
from scipy.stats import skew


class RollingSkewness():
    """
    Custom class to create rolling skewness features.
    """

    def __init__(self, window_sizes, features_names: list = 'rolling_skewness'):
        
        if not isinstance(window_sizes, list):
            window_sizes = [window_sizes]
        self.window_sizes = window_sizes
        self.features_names = features_names

    def transform_batch(self, X: pd.Series) -> pd.DataFrame:
        
        rolling_obj = X.rolling(window=self.window_sizes[0], center=False, closed='left')
        rolling_skewness = rolling_obj.skew()
        rolling_skewness = pd.DataFrame({
                               self.features_names: rolling_skewness
                           }).dropna()

        return rolling_skewness

    def transform(self, X: np.ndarray) -> np.ndarray:
        
        X = X[~np.isnan(X)]
        if len(X) > 0:
            rolling_skewness = np.array([skew(X, bias=False)])
        else:
            rolling_skewness = np.nan
        
        return rolling_skewness

In [9]:
# Custom function to create weights
# ==============================================================================
def custom_weights(index):
    """
    Return 0 if index is between 2004-01-01 and 2005-01-01.
    """
    weights = np.where(
                  (index >= '2004-01-01') & (index <= '2005-01-01'),
                   0,
                   1
              )

    return weights

In [10]:
# Create and train forecaster
# ==============================================================================
window_features = RollingSkewness(window_sizes=3)

forecaster = ForecasterRecursive(
                 regressor       = RandomForestRegressor(random_state=123),
                 lags            = 3,
                 window_features = window_features,
                 weight_func     = custom_weights,
                 forecaster_id   = "forecaster_custom_features"
             )

forecaster.fit(y=data['y'])

<div class="admonition note" name="html-admonition" style="background: rgba(255,145,0,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #ff9100; border-color: #ff9100; padding-left: 10px; padding-right: 10px">

<p class="title">
    <i style="font-size: 18px; color:#ff9100; border-color: #ff1744;"></i>
    <b style="color: #ff9100;"> <span style="color: #ff9100;">&#9888;</span> Warning</b>
</p>

The <code>save_forecaster</code> function will save the functions used to create the weights as a module (<code>custom_weights.py</code>). <b>But the classes used to create the window features will not be saved</b>. Therefore, you must ensure that these classes are available in the environment where the Forecaster is loaded.

</div>

In [11]:
# Save model and custom function
# ==============================================================================
save_forecaster(
    forecaster, 
    file_name = 'forecaster_custom_features.joblib', 
    save_custom_functions = True, 
    verbose = False
)

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\skforecast\utils\utils.py:1898: SaveLoadSkforecastWarning: The Forecaster includes custom user-defined classes in the `window_features` argument. These classes are not saved automatically when saving the Forecaster. Please ensure you save these classes manually and import them before loading the Forecaster.
    Custom classes: RollingSkewness
Visit the documentation for more information: https://skforecast.org/latest/user_guides/save-load-forecaster.html#saving-and-loading-a-forecaster-model-with-custom-features 
 You can suppress this warning using: warnings.simplefilter('ignore', category=SaveLoadSkforecastWarning)
  warnings.warn(


At this point, the `RollingSkewness` class is manually saved in a file called `rolling_skewness.py`. This file must be available in the environment where the Forecaster is loaded.

In [12]:
# Load model and custom function
# ==============================================================================
from rolling_skewness import RollingSkewness  # This has to be generated manually
from custom_weights import custom_weights

forecaster_loaded = load_forecaster('forecaster_custom_features.joblib', verbose=True)

ForecasterRecursive 
Regressor: RandomForestRegressor 
Lags: [1 2 3] 
Window features: ['rolling_skewness'] 
Window size: 3 
Exogenous included: False 
Exogenous names: None 
Transformer for y: None 
Transformer for exog: None 
Weight function included: True 
Differentiation order: None 
Training range: [Timestamp('1991-07-01 00:00:00'), Timestamp('2008-06-01 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: MS 
Regressor parameters: 
    {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth':
    None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None,
    'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100,
    'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0,
    'warm_start': False} 
fit_kwargs: {} 
Creation date: 2024-11-10 17:43:26 
Last fit date: 2024-11-10 17:43:26 
Skforecast version: 0.14